# Alep Tutorial

## Dispersal

Rational

In [1]:
# Python import
import collections

# Wheat simulation
from alinea.adel.astk_interface import AdelWheat

# Lesion 
from alinea.alep.fungus import Lesion, Fungus

# Dispersal Models
from alinea.alep.dispersal_emission import SimpleEmission
from alinea.alep.dispersal_transport import SeptoriaRainDispersal, PowderyMildewWindDispersal, BrownRustDispersal

# Generic algorithm to disperse DispersalUnit on MTG
from alinea.alep.protocol import disperse

# Ploting functions
from alinea.alep.disease_outputs import plot3d_transparency

# PlantGL geometric library
import openalea.plantgl.all as pgl

# MTG selection
from alinea.alep.architecture import get_leaves


ModuleNotFoundError: No module named 'alinea.alep.dispersal_emission'

In [ ]:
%gui qt

### Two generic functions

In [3]:
# utilities to place the source leaf
def is_iterable(obj):
    """ Test if object is iterable """
    return isinstance(obj, collections.Iterable)
    
def get_source_leaf_and_max_height(g, position='center', relative_height=2./3):
    """ Returns the vertex identifier and its height (z ccord).
    
    :Algorithm:
        Select all the leaves.
        Compute the bounding box of each leaf and its center (centroid).
        Compute the maximum height (z max) of all the centroids
        Then, compute the distance (norm) ???
    """
    tesselator = pgl.Tesselator()
    bbc = pgl.BBoxComputer(tesselator)
    leaves = get_leaves(g, label='LeafElement')
    centroids = g.property('centroid')
    geometries = g.property('geometry')
    targets = list(leaf for leaf in leaves if leaf in geometries.iterkeys())
    for vid in targets:
        if is_iterable(geometries[vid]):
            bbc.process(pgl.Scene(geometries[vid]))
        else:
            bbc.process(pgl.Scene([pgl.Shape(geometries[vid])]))
        center = bbc.result.getCenter()
        centroids[vid] = center
    zmax = max(centroids.items(), key=lambda x:x[1][2])[1][2]
    distances = {vid:pgl.norm(centroids[vid]-(0,0,relative_height*zmax)) for vid in centroids}
    if position=='center':
        return min(distances.items(), key=lambda x:x[1])[0], zmax
    elif position=='border':
        return max(distances.items(), key=lambda x:x[1])[0], zmax


## Create your own lesion

In [4]:
fungus_name = "lesion_tutorial"

# Create our own emitting lesion for this example
class LesionTutoDispersal(Lesion):
    def __init__(self, mutable=False):
        super(LesionTutoDispersal, self).__init__()
        self.fungus_name = fungus_name

    def emission(*args, **kwds):
        return 10000


## Create a lesion instance used for the dispersal model

In [5]:
fungus = Fungus(Lesion=LesionTutoDispersal, parameters={"name":fungus_name})
source_lesion = fungus.lesion()


## Create a wheat architectural model with geometry at a given development stage

In [8]:
# create wheat canopy 
adel= AdelWheat(nplants=100, nsect=7)
g = adel.setup_canopy(1500)

# Plot the 3D plants

In [9]:
pgl.Viewer.display(adel.plot(g))

# Select some leaves to set the infectious status

In [10]:
leaf_ids = get_leaves(g, label='LeafElement')

# Deterministic choice 
leaf_index = 1
# Select randomly a leaf
from random import randint
leaf_index = randint(1, len(leaf_ids))-1

source_leaf = leaf_ids[leaf_index]

# Alternatively, pick a leaf based on its position in the canopy
vid, hmax = get_source_leaf_and_max_height(g)
source_leaf = g.node(vid)


# Inoculate the source leaf

In [11]:
# inoculate this leaf
source_lesion.is_sporulating = True # Required for Popdrops usage
source_leaf.lesions = [source_lesion]

# Setup the dispersal models

In [12]:
# Setup dispersal models + parameters for dispersion
emitter = SimpleEmission()
#rain 3d (slow)
transporter = SeptoriaRainDispersal()
# wind 3d (callonec model)
#transporter = PowderyMildewWindDispersal(label='LeafElement')
# wind 1D
#transporter = BrownRustDispersal(domain_area=adel.domain_area)
# missing : rian 1D (septo 3d dispersal model)


# Simulate using the generic algorithm

In [13]:
# Simulate one dispersal event
g = disperse(g, emission_model=emitter, transport_model=transporter, fungus_name=fungus_name)



# Plot the results

In [14]:
# Visualize (currently only works for septoria)
# rain/ wind
transporter.plot_distri_3d(g)
# wind 1D
#transporter.plot_layers(g)
